http://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

In [544]:
from tensorflow.contrib import keras
import tensorflow as tf

import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import psycopg2
from sklearn.model_selection import train_test_split

In [545]:
# fix random seed for reproducibility
np.random.seed(7)

# Load data

In [546]:
data = pd.read_csv('rnn_train/chenmark.csv')

# drop the first contest--too much variance
data.sort_values('ratingupdatetimeseconds', inplace=True)
firstcid = data.contestid.values[0]

#data.reset_index(inplace=True)
data.drop(data.index[data.contestid == firstcid], axis=0, inplace=True)

In [547]:
con = psycopg2.connect(database='codeforces', user='Joy')
cur = con.cursor()

## binarize some variables

In [548]:
cur.execute("select * from all_participanttypes", con)
all_part = [c[1] for c in cur.fetchall()]

cur.execute("select * from all_tags", con)
all_tags = [c[1] for c in cur.fetchall()]

cur.execute("select * from all_language", con)
all_lang = [c[1] for c in cur.fetchall()]

#cur.execute("select * from all_verdicts", con)
#all_verd = [c[1] for c in cur.fetchall()]

In [549]:
# set binary columns to binary, some of them were counts by mistake
bin_vars = all_part + all_tags + all_lang
data[bin_vars] = data[bin_vars].fillna(value=0)

for b in bin_vars:
    data.loc[ data[b] > 0, b] = 1

In [550]:
data.head()

,CHALLENGED,COMPILATION_ERROR,CONTESTANT,CRASHED,GNU C++,GNU C++11,GYM,MEMORY_LIMIT_EXCEEDED,MS C++,Mysterious Language,...,FALSE,Mono C#,Java 7,Tcl,Haskell,Cobol,Io,GNU C++0x,GNU C++14,GNU C++11 ZIP
142,NaN,NaN,1.0,NaN,1.0,0.0,0.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,NaN,NaN,1.0,NaN,1.0,0.0,0.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143,NaN,NaN,1.0,NaN,1.0,0.0,0.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
145,NaN,NaN,1.0,NaN,1.0,0.0,0.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,NaN,NaN,1.0,NaN,1.0,0.0,0.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [551]:
for c in data.columns:
    print c

CHALLENGED
COMPILATION_ERROR
CONTESTANT
CRASHED
GNU C++
GNU C++11
GYM
MEMORY_LIMIT_EXCEEDED
MS C++
Mysterious Language
OK
OUT_OF_COMPETITION
PARTIAL
PRACTICE
PyPy 2
Python 2
REJECTED
RUNTIME_ERROR
SKIPPED
TIME_LIMIT_EXCEEDED
VIRTUAL
WRONG_ANSWER
contestid
delta_smoothed_1months
delta_smoothed_2months
delta_smoothed_3months
delta_smoothed_4months
delta_smoothed_5months
handle
index
newrating
oldrating
points
problem_rating
rank
ratingupdatetimeseconds
smoothed_1months
smoothed_2months
smoothed_3months
smoothed_4months
smoothed_5months
solvetimeseconds
starttimeseconds
stoptimeseconds
shortest paths
2-sat
greedy
meet-in-the-middle
matrices
number theory
constructive algorithms
chinese remainder theorem
implementation
ternary search
schedules
dsu
dfs and similar
graph matchings
string suffix structures
math
probabilities
fft
divide and conquer
two pointers
trees
data structures
flows
sortings
expression parsing
dp
hashing
bitmasks
*special
geometry
combinatorics
graphs
brute force
games
b

##  Remove unnecessary columns

In [552]:
month = 3

In [553]:
df_data = data
for m in range(1,6):
    if m == month:
        continue
    name1 = "delta_smoothed_%dmonths" % m
    name2 = "smoothed_%dmonths" % m
    
    df_data.drop([name1, name2], axis=1, inplace=True)

df_train = df_data.drop(['handle', 'index'], axis=1)

In [554]:
df_train.fillna(value=0, inplace=True)

## Feature scaling and grouping by contest

In [555]:
colname = 'delta_smoothed_%dmonths' % month
cols = list(df_train.columns.values)
colidx = cols.index(colname)

cids = df_train.contestid

scaler = MinMaxScaler(feature_range=(0, 1))
df_train_scaled = scaler.fit_transform(df_train)
df_train_scaled = pd.DataFrame(df_train_scaled)
df_train_scaled.columns = cols

# add back in cols that should not be scaled
df_train_scaled['contestid'] = cids
df_train_scaled[colname] = df_train[colname]

In [556]:
groups = df_train_scaled.groupby('contestid')

In [557]:
trainlist = []
ylist = []
bins = range(-200, 200, 20)

for k, v in groups:
    base = [0] * (len(bins) + 1)
    v.is_copy = False
    
    v.drop('contestid', axis=1, inplace=True)
    y = v.loc[:, colname].values[0]
    v.drop(colname, inplace=True, axis=1)
    
    trainlist.append(v)
    ylist.append(y)

In [558]:
min(ylist), max(ylist)

(-23.699999999999999, 65.314285709999993)

In [584]:
yvecs = [ [0] * (len(bins) + 1) for i in range(len(ylist))]
idx1 = np.digitize(ylist, bins=bins)

for i, j in enumerate(idx1):
    yvecs[i][j] = 1
#yvecs

In [560]:
ary = np.array(yvecs)

In [561]:
len(trainlist), len(ylist)

(88, 88)

## Pad X values

In [562]:
# TODO: need to make this "universal" across all users
maxtimepts = max([len(t) for t in trainlist])
size = trainlist[0].shape[1]

for i in range(len(trainlist)):
    gap = maxtimepts - len(trainlist[i])
    for j in range(gap):
        nullrow = [0] * size
        trainlist[i].loc[-j-1] = nullrow
    trainlist[i].sort_index(inplace = True)

In [583]:
#print len(trainlist)
#[t.shape for t in trainlist]

In [564]:
dfx = pd.concat(trainlist)
dfx.reset_index(inplace=True, drop=True)

In [565]:
arx = np.array(dfx)
arx = np.reshape(arx, (len(trainlist), maxtimepts, 111))

# Set up keras model
https://keras.io/layers/recurrent/
https://keras.io/getting-started/sequential-model-guide/

```keras.layers.recurrent.Recurrent(return_sequences=False, go_backwards=False, stateful=False, unroll=False, implementation=0)```
* ```weights```: list of Numpy arrays to set as initial weights. The list should have 3 elements, of shapes: [(input_dim, output_dim), (output_dim, output_dim), (output_dim,)].
* ```return_sequences```: Boolean. Whether to return the last output in the output sequence, or the full sequence.
* ```go_backwards```: Boolean (default False). If True, process the input sequence backwards and return the reversed sequence.
* ```stateful```: Boolean (default False). If True, the last state for each sample at index i in a batch will be used as initial state for the sample of index i in the following batch.
* ```unroll```: Boolean (default False). If True, the network will be unrolled, else a symbolic loop will be used. Unrolling can speed-up a RNN, although it tends to be more memory-intensive. Unrolling is only suitable for short sequences.
* ```implementation```: one of {0, 1, or 2}. If set to 0, the RNN will use an implementation that uses fewer, larger matrix products, thus running faster on CPU but consuming more memory. If set to 1, the RNN will use more matrix products, but smaller ones, thus running slower (may actually be faster on GPU) while consuming less memory. If set to 2 (LSTM/GRU only), the RNN will combine the input gate, the forget gate and the output gate into a single matrix, enabling more time-efficient parallelization on the GPU.
    * Note: RNN dropout must be shared for all gates, resulting in a slightly reduced regularization.
* ```input_dim```: dimensionality of the input (integer). This argument (or alternatively, the keyword argument input_shape) is required when using this layer as the first layer in a model.
* ```input_length```: Length of input sequences, to be specified when it is constant. This argument is required if you are going to connect  Flatten then Dense layers upstream (without it, the shape of the dense outputs cannot be computed). Note that if the recurrent layer is not the first layer in your model, you would need to specify the input length at the level of the first layer (e.g. via the input_shape argument)

**Note on using statefulness in RNNs**

You can set RNN layers to be 'stateful', which means that the states computed for the samples in one batch will be reused as initial states for the samples in the next batch. This assumes a one-to-one mapping between samples in different successive batches.

To enable statefulness:
- specify ```stateful=True``` in the layer constructor. 
- specify a fixed batch size for your model, by passing if sequential model:  ```batch_input_shape=(...)``` to the first layer in your model.
else for functional model with 1 or more Input layers:  ```batch_shape=(...)``` to all the first layers in your model.
This is the expected shape of your inputs including the batch size. It should be a tuple of integers, e.g. (32, 10, 100).
- specify ```shuffle=False``` when calling ```fit()```.

To reset the states of your model, call ```.reset_states()``` on either a specific layer, or on your entire model.

# NOTE CURRENTLY USING THE WRONG OUTPUT LAYER, NEED REGRESSION NOT CLASSIFICATION

In [566]:
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import GRU

In [652]:
def get_user_data(user):
    # -----------------------------
    # Load data
    data = pd.read_csv('rnn_train/%s.csv'%user)

    # drop the first contest--too much variance
    data.sort_values('ratingupdatetimeseconds', inplace=True)
    firstcid = data.contestid.values[0]
    data.drop(data.index[data.contestid == firstcid], axis=0, inplace=True)

    # -----------------------------
    # binarize some variables

    cur.execute("select * from all_participanttypes", con)
    all_part = [c[1] for c in cur.fetchall()]

    cur.execute("select * from all_tags", con)
    all_tags = [c[1] for c in cur.fetchall()]

    cur.execute("select * from all_language", con)
    all_lang = [c[1] for c in cur.fetchall()]

    # set binary columns to binary, some of them were counts by mistake
    bin_vars = all_part + all_tags + all_lang
    data[bin_vars] = data[bin_vars].fillna(value=0)

    for b in bin_vars:
        data.loc[ data[b] > 0, b] = 1

    # -----------------------------
    # remove information for other months
    df_data = data
    for m in range(1,6):
        if m == month:
            continue
        name1 = "delta_smoothed_%dmonths" % m
        name2 = "smoothed_%dmonths" % m
        
        df_data.drop([name1, name2], axis=1, inplace=True)

    df_train = df_data.drop(['handle', 'index'], axis=1)
    df_train.fillna(value=0, inplace=True)


    # -----------------------------
    # Feature scaling and grouping by contest
    colname = 'delta_smoothed_%dmonths' % month
    cols = list(df_train.columns.values)
    colidx = cols.index(colname)

    cids = df_train.contestid

    scaler = MinMaxScaler(feature_range=(0, 1))
    df_train_scaled = scaler.fit_transform(df_train)
    df_train_scaled = pd.DataFrame(df_train_scaled)
    df_train_scaled.columns = cols

    # add back in cols that should not be scaled
    df_train_scaled['contestid'] = cids
    df_train_scaled[colname] = df_train[colname]

    groups = df_train_scaled.groupby('contestid')

    # -----------------------------
    # create list of inputs for training
    trainlist = []
    ylist = []

    for k, v in groups:
        base = [0] * (len(bins) + 1)
        v.is_copy = False
        
        v.drop('contestid', axis=1, inplace=True)
        y = v.loc[:, colname].values[0]
        v.drop(colname, inplace=True, axis=1)
        
        trainlist.append(v)
        ylist.append(y)


    yvecs = [ [0] * (len(bins) + 1) for i in range(len(ylist))]
    idx1 = np.digitize(ylist, bins=bins)
    for i, j in enumerate(idx1):
        yvecs[i][j] = 1
    ary = np.array(yvecs)


    # -----------------------------
    # Pad X values
    # TODO: need to make this "universal" across all users
    #maxtimepts = max([len(t) for t in trainlist])
    size = trainlist[0].shape[1]

    for i in range(len(trainlist)):
        gap = maxtimepts - len(trainlist[i])
        for j in range(gap):
            nullrow = [0] * size
            trainlist[i].loc[-j-1] = nullrow
        trainlist[i].sort_index(inplace = True)


    dfx = pd.concat(trainlist)
    dfx.reset_index(inplace=True, drop=True)

    arx = np.array(dfx)
    arx = np.reshape(arx, (len(trainlist), maxtimepts, 111))
    return arx, ary



In [687]:
xx_train, yy_train = get_user_data("lewin")

xx_test, yy_test = get_user_data("chenmark")

In [678]:
maxtimepts = 110 #max([len(t) for t in trainlist])
size = 111 #trainlist[0].shape[1]
n_neurons = 10

print maxtimepts, size, len(bins)

110 111 20


In [707]:
def create_model(layer1, layer2, batch_input_shape):
    model = Sequential()
    batch_input_shape = (xx_train.shape[0], maxtimepts, size)
    model.add(GRU(layer1, return_sequences=True, stateful=True, batch_input_shape=batch_input_shape))
    model.add(Dropout(0.5))
    model.add(GRU(layer2, return_sequences=False, stateful=True, batch_input_shape=batch_input_shape))
    model.add(Dropout(0.5))
    model.add(Dense(len(bins) + 1, activation='softmax'))
    return model

In [712]:
neurons1 = 4
neurons2 = 1
model = create_model(neurons1, neurons2, batch_input_shape)

In [699]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_25 (GRU)                 (49, 10)                  3660      
_________________________________________________________________
dropout_45 (Dropout)         (49, 10)                  0         
_________________________________________________________________
dense_47 (Dense)             (49, 21)                  231       
Total params: 3,891
Trainable params: 3,891
Non-trainable params: 0
_________________________________________________________________


In [713]:
# For a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [714]:
print batch_input_shape
print model.output_shape

print xx_train.shape
print yy_train.shape

(88, 110, 111)
(49, 21)
(49, 110, 111)
(49, 21)


In [715]:
#fit(self, x, y, batch_size=32, epochs=10, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0)
#model.fit(arx, ary, epochs=50, batch_size=len(trainlist), shuffle=False)
model.fit(xx_train, yy_train, epochs=100, batch_size=xx_train.shape[0], shuffle=False)
#score = model.evaluate(x_test, y_test, batch_size=len(trainlist))

Epoch 1/100
49/49 [==============================] - 6s - loss: 3.0536 - acc: 0.0000e+00
Epoch 2/100
49/49 [==============================] - 0s - loss: 3.0434 - acc: 0.1837
Epoch 3/100
49/49 [==============================] - 0s - loss: 3.0525 - acc: 0.1429
Epoch 4/100
49/49 [==============================] - 0s - loss: 3.0365 - acc: 0.1429
Epoch 5/100
49/49 [==============================] - 0s - loss: 3.0273 - acc: 0.1633
Epoch 6/100
49/49 [==============================] - 0s - loss: 3.0365 - acc: 0.1224
Epoch 7/100
49/49 [==============================] - 0s - loss: 3.0283 - acc: 0.1224
Epoch 8/100
49/49 [==============================] - 0s - loss: 3.0271 - acc: 0.1837
Epoch 9/100
49/49 [==============================] - 0s - loss: 3.0397 - acc: 0.1633
Epoch 10/100
49/49 [==============================] - 0s - loss: 3.0324 - acc: 0.0816
Epoch 11/100
49/49 [==============================] - 0s - loss: 3.0244 - acc: 0.1429
Epoch 12/100
49/49 [==============================] - 0s - 

49/49 [==============================] - 0s - loss: 2.9110 - acc: 0.2041
Epoch 97/100
49/49 [==============================] - 0s - loss: 2.9058 - acc: 0.1429
Epoch 98/100
49/49 [==============================] - 0s - loss: 2.9012 - acc: 0.2041
Epoch 99/100
49/49 [==============================] - 0s - loss: 2.9027 - acc: 0.2041
Epoch 100/100
49/49 [==============================] - 0s - loss: 2.9076 - acc: 0.1633


In [703]:
newmodel = create_model(neurons1, neurons2, batch_input_shape)
old_weights = model.get_weights()
newmodel.set_weights(old_weights)

newmodel.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [704]:
score = newmodel.evaluate(xx_test, yy_test, batch_size=xx_test.shape[0])

print model.metrics_names
score

88/88 [==============================] - 2s
['loss', 'acc']


[2.4162671566009521, 0.35227271914482117]

Looks like we overfit the crap out of that one. Let's look at the number of parameters in this system

# Junk

In [224]:
# For a single-input model with 10 classes (categorical classification):

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Generate dummy data
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(10, size=(1000, 1))

# Convert labels to categorical one-hot encoding
one_hot_labels = keras.utils.to_categorical(labels, num_classes=10)

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, one_hot_labels, epochs=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 4s - loss: 2.3257 - acc: 0.0960     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 2.3010 - acc: 0.1080     - ETA: 0s - loss: 2.3050 - acc: 0.10
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 2.2916 - acc: 0.1280       
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 2.2832 - acc: 0.1400     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 2.2743 - acc: 0.1520     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 2.2680 - acc: 0.1650     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 2.2594 - acc: 0.1730     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 2.2527 - acc: 0.1740     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 2.2473 - acc: 0.1850     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 2.2401 - acc: 0.1830     


In [13]:
'''
A Recurrent Neural Network (LSTM) implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)
Long Short Term Memory paper: http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

'''
To classify images using a recurrent neural network, we consider every image
row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then
handle 28 sequences of 28 steps for every sample.
'''

# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}


def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Iter 1280, Minibatch Loss= 2.090559, Training Accuracy= 0.24219
Iter 2560, Minibatch Loss= 1.680801, Training Accuracy= 0.39844
Iter 3840, Minibatch Loss= 1.353599, Training Accuracy= 0.49219
Iter 5120, Minibatch Loss= 1.229820, Training Accuracy= 0.57031
Iter 6400, Minibatch Loss= 1.041029, Training Accuracy= 0.64062
Iter 7680, Minibatch Loss= 0.937305, Training Accuracy= 0.69531
Iter 8960, Minibatch Loss= 0.710454, Training Accuracy= 0.72656
Iter 10240, Minibatch Loss= 0.571756, Training Accuracy= 0.84375
Iter 11520, Minibatch Loss= 0.564859, Tr

## Split dataset (not relevant yet)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.33)

In [ ]:
print X_train.shape
print y_train.shape

In [697]:
### as the first layer in a Sequential model
##model = Sequential()
##model.add(LSTM(32, input_shape=(10, 64)))
### now model.output_shape == (None, 32)
### note: `None` is the batch dimension.
##
### for subsequent layers, no need to specify the input size:
##model.add(LSTM(16))
#
## to stack recurrent layers, you must use return_sequences=True
## on any recurrent layer that feeds into another recurrent layer.
## note that you only need to specify the input size on the first layer.
#model = Sequential()
##model.add(Embedding(max_features, output_dim=50))
#
#batch_input_shape = (xx_train.shape[0], maxtimepts, size)
#model.add(GRU(n_neurons, return_sequences=False, stateful=True, batch_input_shape=batch_input_shape))
#model.add(Dropout(0.5))
##model.add(LSTM(32, return_sequences=False, stateful=True))
##model.add(Dropout(0.5))
#model.add(Dense(len(bins) + 1, activation='softmax'))